In [1]:
#多线程 对象
import threading

class CThread (threading.Thread):
    def __init__(self, n):
        threading.Thread.__init__(self)  # 重写父类方法
        self.num = n

    def run(self):
        print(self.num)

if __name__ == "__main__":
    a = CThread(1)  # 开启一个线程
    a.start()       # 启动线程
	
	# 开启多个线程
    # for i in range(6): 
    #	a = CThread(i)  # 连续开启多个线程
    #	a.start()       # 启动线程

1


In [4]:
#停止线程 对象 内部变量Flag 退出标记 
import threading
import time

class MyThread(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.Flag = True  # 停止标志位
        self.Parm = 0  # 用来被外部访问的

    def run(self):
        while (True):
            if (not self.Flag):
                break
            else:
                time.sleep(2)

    def setFlag(self, parm):  # 外部停止线程的操作函数
        self.Flag = parm  # boolean

    def setParm(self, parm):  # 外部修改内部信息函数
        self.Parm = parm

    def getParm(self):  # 外部获得内部信息函数
        return self.Parm


if __name__ == "__main__":
    testThread = MyThread()
    testThread.setDaemon(True)  # 设为保护线程，主进程结束会关闭线程

    testThread.start()  # 开始线程

    time.sleep(2)  # 主进程休眠 2 秒
    print(testThread.is_alive())  # 查看线程运行状态
    
    testThread.setFlag(False)  # 修改线程运行状态
    time.sleep(2)
    print(testThread.is_alive())  # 查看线程运行状态


True
False


In [ ]:
#停止线程 对象 内部Event 退出标记 
# encoding:utf-8
import time
import threading

class StoppableThread(threading.Thread):
    """Thread class with a stop() method. The thread itself has to check
    regularly for the stopped() condition."""

    def __init__(self,  *args, **kwargs):
        super(StoppableThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def stopped(self):
        return self._stop_event.is_set()

    def run(self):
        print("begin run the child thread")
        while True:
            print("sleep 1s")
            time.sleep(1)
            if self.stopped():
                
                break

if __name__ == "__main__":
    print("begin run main thread")
    t = StoppableThread()
    t.start()
    time.sleep(3)
    t.stop()
    print("main thread end")

In [1]:
#多线程 threading.Thread
import threading
import time


def download(i):
    print('开始下载文件%d' % i)
    time.sleep(5)
    print('文件下载完成')


if __name__ == '__main__':

    for i in range(5):  # 利用循环创建5个线程
        t = threading.Thread(target=download, args=(i,))
        print(len(threading.enumerate()))  # 查看线程数量和进程数量总和
        # 启动线程
        t.start()
        t.join()


7
开始下载文件0
文件下载完成
7
开始下载文件1
文件下载完成
7
开始下载文件2
文件下载完成
7
开始下载文件3
文件下载完成
7
开始下载文件4
文件下载完成


In [1]:
# 停止线程 使用ctypes强行杀掉线程 ctypes.pythonapi.PyThreadState_SetAsyncExc
import inspect
import ctypes
import threading

def terminate_thread(thread):
    if not thread.isAlive():
        return

    exc = ctypes.py_object(SystemExit)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(
        ctypes.c_long(thread.ident), exc)
    if res == 0:
        raise ValueError("nonexistent thread id")
    elif res > 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(thread.ident, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")


def _async_raise(tid, exctype):

    tid = ctypes.c_long(tid)

    if not inspect.isclass(exctype):
        exctype = type(exctype)

    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(
        tid, ctypes.py_object(exctype))

    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")


def stop_thread(thread):
    if not thread.is_alive():
        return
    _async_raise(thread.ident, SystemExit)


In [7]:
#多线程 定时器 Timer
# t = threading.Timer(6, download, (11,) )
# t.start()
# t.join()

# stop_thread(t)

for thread in threading.enumerate():
    print(f'{thread.getName() = }')
    # if thread.getName() != 'MainThread':
        # stop_thread(thread)

thread.getName() = 'MainThread'
thread.getName() = 'Heartbeat'
thread.getName() = 'Thread-2'


In [ ]:
#多线程 锁对象 threading.Lock
import threading
import time


def run(n):
    lock.acquire()
    print("task:", n, threading.current_thread())
    lock.release()


lock = threading.Lock()

start_time = time.time()

t_obj = []
for i in range(50):
    t = threading.Thread(target=run, args=("t-%d" % i,))
    # 把当前线程设置为守护线程,当主线程执行完毕则关闭所有的子线程
    # t.setDaemon(True)
    t.start()
    # 为了防止不阻塞后面的子进程启动,不在这里加join,先放到一个列表中
    t_obj.append(t)

# 可以保证所有的子进程都进行完毕了再进入主进程
for t in t_obj:
    t.join()

# 主线程
# threading.activeCount() 当前活跃的进程数
print("----all thread has finished----",
      threading.current_thread(), threading.activeCount())
print("cost:", time.time() - start_time)


In [9]:
#线程 递归锁对象：threading.RLock
import threading
rLock = threading.RLock() 

rLock.acquire()
rLock.acquire() 

rLock.release()
rLock.release()

In [ ]:
#多线程 递归锁对象：threading.RLock
import threading, time
def run1():
    print("grab the first part data")
    lock.acquire()
    global num
    num += 1
    lock.release()
    return num
def run2():
    print("grab the second part data")
    lock.acquire()
    global num2
    num2 += 1
    lock.release()
    return num2
def run3():
    lock.acquire()
    res = run1()
    print('--------between run1 and run2-----')
    res2 = run2()
    lock.release()
    print(res, res2)

num, num2 = 0, 0
# 递归锁防止锁死
lock = threading.RLock()
for i in range(10):
    t = threading.Thread(target=run3)
    t.start()

while threading.active_count() != 1:
    print(threading.active_count())
else:
    print('----all threads done---')
    print(num, num2)

In [ ]:
#多线程 信号量 BoundedSemaphore
import threading,time

def run(n):
    semaphore.acquire()
    time.sleep(1)
    print("run the thread: %s\n" % n)
    semaphore.release()

semaphore = threading.BoundedSemaphore(5)

if __name__ == "__main__":
    semaphore = threading.BoundedSemaphore(5)  # 最多允许5个线程同时运行
    for i in range(22):
        t = threading.Thread(target=run, args=(i,))
        t.start()

In [ ]:
#线程 事件 Event
import random
import signal
import threading
import time

exit_event = threading.Event()

def bg_thread():
    # for i in range(1, 30):
    #     print(f'{i} of 30 iterations...')
    #     time.sleep(random.random())  # do some work...
    #     if exit_event.is_set():
    #         break

    for i in range(1, 30):  
        print(f'{i} of 30 iterations...')  
        if exit_event.wait(timeout=random.random()):  
            break

    print(f'{i} iterations completed before exiting.')  

def signal_handler(signum, frame):  
    exit_event.set()  

signal.signal(signal.SIGINT, signal_handler)  
th = threading.Thread(target=bg_thread)  
th.start()  
th.join()

In [ ]:
#多线程 Event
import threading,time
event = threading.Event()
def lighter():
    count = 0
    # 先设置路灯
    event.set()
    while True:
        # 红灯
        if count >= 5 and count < 10:
            event.clear()
            print("\033[41;1mred light is on...\033[0m")
        elif count > 10:
            event.set()
            count = 0
        else:
            print("\033[42;1mgreen light is on...\033[0m")
        time.sleep(1)
        count += 1

def car(name):
    while True:
        if event.is_set():
            print("[%s] running..." % name)
            time.sleep(1)
        else:
            print("[%s] sees red light , waiting...." % name)
            event.wait()
            print("\033[34;1m[%s] green light is on, start going...\033[0m" % name)

light = threading.Thread(target=lighter,)
light.start()

car = threading.Thread(target=car,args=("长安",))
car.start()

In [5]:
#随机数 random
import random 
import time

print(f'{time.strftime("%Y-%m-%d %H:%M:%S")} **********')
ran = random.random()
%time time.sleep(ran)
print(f'{time.strftime("%Y-%m-%d %H:%M:%S")} *****{ran = }*****')

2022-09-07 00:29:11 **********
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 738 ms
2022-09-07 00:29:11 *****ran = 0.7370572832008336*****


In [8]:
#多线程 Condition
import threading
import time

def Seeker(cond, name):
    time.sleep(2)
    cond.acquire()
    print('%s :我已经把眼睛蒙上了！' % name)
    cond.notify()
    cond.wait()
    for i in range(3):
        print('%s is finding!!!' % name)
        time.sleep(2)
    cond.notify()
    cond.release()
    print('%s :我赢了！' % name)

def Hider(cond, name):
    cond.acquire()
    cond.wait()
    for i in range(2):
        print('%s is hiding!!!' % name)
        time.sleep(3)
    print('%s :我已经藏好了，你快来找我吧！' % name)
    cond.notify()
    cond.wait()
    cond.release()
    print('%s :被你找到了，唉~^~!' % name)

if __name__ == '__main__':
    cond = threading.Condition()
    seeker = threading.Thread(target=Seeker, args=(cond, 'seeker'))
    hider = threading.Thread(target=Hider, args=(cond, 'hider'))
    seeker.start()
    hider.start()
    seeker.join()
    hider.join()


seeker :我已经把眼睛蒙上了！
hider is hiding!!!
hider is hiding!!!
hider :我已经藏好了，你快来找我吧！
seeker is finding!!!
seeker is finding!!!
seeker is finding!!!
seeker :我赢了！hider :被你找到了，唉~^~!



In [11]:
for thread in threading.enumerate():
    print(f'{thread.getName() = }')
    
    if thread.getName() != 'MainThread':
        stop_thread(thread)

thread.getName() = 'MainThread'
thread.getName() = 'Heartbeat'
thread.getName() = 'Thread-2'
thread.getName() = 'Thread-7'


In [ ]:
#消息队列 Queue
from multiprocessing import Queue
if __name__ == '__main__':
    q = Queue(3)  # 初始化一个Queue对象，最多可接收3条put消息
    q.put('消息1')
    q.put('消息2')
    print(q.full())
    q.put('消息3')
    print(q.full())
    try:
        q.put('消息4', True, 2)
    except:
        print('消息队列已满，现有消息数量%d' % q.qsize())

    try:
        q.put_nowait()
    except:
        print('消息队列已满，现有消息数量%d' % q.qsize())
    # 读取消息时，先判断消息队列是否为空

    if not q.empty():
        print('从队列中获取信息')
        for i in range(q.qsize()):
            print(q.get_nowait())
        # 先判断消息队列是否已满，在写入
        if not q.full():
            q.put_nowait('消息4')


In [ ]:
# 使用队列在进程间通信 Process Queue
from multiprocessing import Process, Queue
import time
# 向队列中写入数据


def write_task(q):  # 一定要将q传进去
    if not q.full():
        for i in range(5):
            message = '消息'+str(i)
            q.put(message)
            print('写入：%s' % message)
# 从队列读取数据


def read_task(q):  # 一定要将q传进去
    time.sleep(1)
    while not q.empty():
        print('读取：%s' % q.get(True, 2))


if __name__ == '__main__':
    print('父进程开始')
    q = Queue()
    pw = Process(target=write_task, args=(q,))
    pr = Process(target=read_task, args=(q,))
    pw.start()
    pr.start()
    pw.join()
    pr.join()
    print('父进程结束')


In [ ]:
#多线程 queue
import queue,time,threading

q = queue.Queue(maxsize=10)

def Product(name):
    count = 1
    while True:
        q.put("骨头%s" % count)
        print("生产了骨头", count)
        count += 1
        time.sleep(0.1)

def Consumer(name):
    # while q.qsize() > 0:
    while True:
        print("[%s] 取到[%s] 并且吃了它..." % (name, q.get()))
        time.sleep(0.5)

p1 = threading.Thread(target=Product,args=("bob",))
c1 = threading.Thread(target=Consumer,args=("andy",))
c2 = threading.Thread(target=Consumer,args=("james",))

p1.start()
c1.start()
c2.start()

In [3]:
#加密 哈希werkzeug.security 用于Flask登录验证模块

from werkzeug.security import generate_password_hash, check_password_hash
 
#generate_password_hash:哈希加盐
users=[
     {'username': 'Tom', 'password': generate_password_hash('111111')},
     {'username': 'Michael', 'password': generate_password_hash('123456')}
]
 
print(users)
bools=check_password_hash(users[0]['password'], "111111") #True or False
print(bools)


[{'username': 'Tom', 'password': 'pbkdf2:sha256:260000$OrRuUz0kDbqz7PGf$da91f8603e8abb95f3bc31867b98264deaaf54632afc1b3923e5d06a35a6d5df'}, {'username': 'Michael', 'password': 'pbkdf2:sha256:260000$WIRsMu2yePJLgUUf$66b8d285a9f0382d038a68a29773806ec5f97628a9c6552c12d4e59de3a9441f'}]
True


In [5]:
#加密 AES加密解密 Crypto.Cipher Crypto.Util.Padding 用于应用后端与其他项目交互传数据的时候加密解密
# window: pip install pycryptodome
# Linux: pip install pycrypto

import json
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
import hashlib,base64

#key的长度必须为16位,不然报错
AES_KEY="!@qw#$ert%^y&*u("

class AES_Convert:
    def __init__(self):
        self.key = AES_KEY

    #aes 加密
    def Encryption(self,aes_str):
        # 使用key,选择加密方式
        aes = AES.new(self.key.encode('utf-8'), AES.MODE_ECB)
        pad_pkcs7 = pad(aes_str.encode('utf-8'), AES.block_size, style='pkcs7')  # 选择pkcs7补全
        encrypt_aes = aes.encrypt(pad_pkcs7)
        # 加密结果
        encrypted_text = str(base64.encodebytes(encrypt_aes), encoding='utf-8')  # 解码
        encrypted_text_str = encrypted_text.replace("\n", "")
        return encrypted_text_str

    # aes解码
    def Decryption(self, aes_str):
        aes = AES.new(self.key.encode('utf-8'), AES.MODE_ECB)
        # 优先逆向解密base64成bytes
        base64_decrypted = base64.decodebytes(aes_str.encode(encoding='utf-8-sig'))
        decrypted_text = str(aes.decrypt(base64_decrypted), encoding='utf-8-sig').replace('', '')
        # 注意这里解密会产生乱码，所以加密数据时可以用正则匹配掉乱码，具体问题具体对待
        # 这里传输的是json字符可以用这种方式匹配
        # print(decrypted_text)
        # decrypted_text=re.findall('.*\}',decrypted_text)[0]
        # decrypted_text = re.findall('.*\]', decrypted_text)[0]
        return decrypted_text

if __name__ == '__main__':
    datalist=[
        {"username":"Ton","password":"Ton@.com"},
        {"username":"Michael","password":"Michael@.com"},
    ]
    aes_str=json.dumps(datalist)
    EncryptionData = AES_Convert().Encryption(aes_str)
    DecryptionData = AES_Convert().Decryption(EncryptionData)
    print("加密数据："+EncryptionData)
    print("加密解密："+DecryptionData)


加密数据：7+S0Fk5WHEZjOqbAYV/x3CpJ0lg6kTJU8RKRc2+CsR8rY/1Q8Mw9DrAAVGx/y0/iYoO/0l9Yb6wUdH11fuEHvZG6S+zQdrTN/flF2qxq383hyNETNW7ZCZXv2bTtEaG4z9noeXj6SgH3lhNTX5hZOw==
加密解密：[{"username": "Ton", "password": "Ton@.com"}, {"username": "Michael", "password": "Michael@.com"}]


In [ ]:
#JWT加密解密 用于应用前后端认证时的token

import jwt,datetime
SECRECT_KEY="!@qw#$ert%^y&*u("
 
class Jwt:
    def __init__(self):
        self.SECRECT_KEY = SECRECT_KEY
 
 
    # 生成jwt 信息,加密
    def jwtEncoding(self,username=None,id=None, some='xxxx'):
        datetimes=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        option = {
            "time":datetimes,
            "username":username,
            "id":id,
            "some":some
        }
        jwtencode = jwt.encode(option, self.SECRECT_KEY, algorithm='HS256')
        return jwtencode
 
 
    # 解密jwt 信息
    def jwtDecoding(self,token):
        try:
            decoded = jwt.decode(token, self.SECRECT_KEY, algorithms=['HS256'])
        except jwt.ExpiredSignatureError:
            decoded = {"error_msg": "is timeout !!", "some": None}
        except Exception:
            decoded = {"error_msg": "noknow exception!!", "some": None}
        return decoded
 
if __name__ == '__main__':
    jwtencode = Jwt().jwtEncoding(username='admin', id=1)
    print(jwtencode)
    jwtdeode = Jwt().jwtDecoding(jwtencode)
    print(jwtdeode)
